In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from sklearn.preprocessing import PolynomialFeatures, normalize
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

from helpers.feature_extractors import extract_features, extract_features_2d, extract_features_edge, extract_features_cogrey
from helpers.metric_helpers import compute_true_positive_rate
from helpers.visualization_helpers import pretty_confusion, label_to_img
from helpers.dataset_preprocessing import build_model_data, load_image, create_dataset, extract_patches, compute_input_features, compute_output_features

import matplotlib.pyplot as plt
from helpers.temp_helpers import *

from PIL import Image


Using TensorFlow backend.


In [2]:
ROOT_DIR = "training/"
TRAIN_FRACTION = 0.8
FOREGROUND_THRESHOLD = 0.25
patch_size = 16
width = 400
height = 400
n_img = 10

# Extract 2d features

In [3]:
X1, Y1 = build_model_data(ROOT_DIR, extract_features_2d, patch_size=patch_size, n_img=n_img)
X1 = normalize(X1)
scaler = StandardScaler()
scaler.fit(X1)  
X1 = scaler.transform(X1)
mlp1 = MLPClassifier()
mlp1.fit(X1, Y1)

Original loaded dataset size: 10
X [(6250, 2)] and Y [(6250,)]


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

# Extract 6d features

In [4]:
X2, Y2 = build_model_data(ROOT_DIR, extract_features, patch_size=patch_size, n_img=n_img)
X2 = normalize(X2)
scaler = StandardScaler()
scaler.fit(X2)  
X2 = scaler.transform(X2)
mlp2 = MLPClassifier()
mlp2.fit(X2, Y2)

Original loaded dataset size: 10
X [(6250, 6)] and Y [(6250,)]


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

# Add canny edge detector

In [5]:
X3, Y3 = build_model_data(ROOT_DIR, extract_features_edge, patch_size=patch_size, n_img=n_img)
X3 = normalize(X3)
scaler = StandardScaler()
scaler.fit(X3)  
X3 = scaler.transform(X3)
mlp3 = MLPClassifier()
mlp3.fit(X3, Y3)

Original loaded dataset size: 10
X [(6250, 7)] and Y [(6250,)]


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

# Add polynomial features

In [6]:
X4, Y4 = build_model_data(ROOT_DIR, extract_features_edge, patch_size=patch_size, n_img=n_img)
poly = PolynomialFeatures(3)
X4 = poly.fit_transform(X4)
X4 = normalize(X4)
scaler = StandardScaler()
scaler.fit(X4)  
X4 = scaler.transform(X4)
mlp4 = MLPClassifier()
mlp4.fit(X4, Y4)

Original loaded dataset size: 10
X [(6250, 7)] and Y [(6250,)]


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

# Comparison

In [7]:
models = [(mlp1, X1, Y1), (mlp2, X2, Y2), (mlp3, X3, Y3), (mlp4, X4, Y4)]

for m in models:
    # Predict on the training set
    Z = m[0].predict(m[1])

    TPR = compute_true_positive_rate(m[2],Z)
    print('True positive rate = ' + str(TPR))
    print('F1-score:', f1_score(m[2],Z))
    pretty_confusion(["road", "bg"], m[2], Z)
    print('\n')

True positive rate = 0.05904
F1-score: 0.291469194313
            t/p           road             bg
           road           4087            335
             bg           1459            369


True positive rate = 0.17312
F1-score: 0.650631389056
            t/p           road             bg
           road           4006            416
             bg            746           1082


True positive rate = 0.1664
F1-score: 0.629349470499
            t/p           road             bg
           road           3985            437
             bg            788           1040


True positive rate = 0.16272
F1-score: 0.648596938776
            t/p           road             bg
           road           4131            291
             bg            811           1017




# Cross-validation without post-processing

In [8]:
for m in models:
    cr_val = cross_val_score(m[0], m[1], m[2], cv=10, scoring='accuracy')
    print('ACCURACY')
    print(cr_val)
    print('mean accuracy:', cr_val.mean())
    cr_val = cross_val_score(m[0], m[1], m[2], cv=10, scoring='f1')
    print('F1-SCORE')
    print(cr_val)
    print('mean f1:', cr_val.mean())
    print('\n')

ACCURACY
[ 0.71086262  0.71884984  0.7744      0.688       0.6944      0.6784
  0.7248      0.7168      0.70352564  0.72115385]
mean accuracy: 0.713119194724
F1-SCORE
[ 0.22222222  0.34328358  0.44787645  0.24902724  0.29893238  0.26470588
  0.18269231  0.21524664  0.21757322  0.25108225]
mean f1: 0.269264217354


ACCURACY
[ 0.83386581  0.85463259  0.8448      0.7568      0.8         0.7504
  0.8064      0.808       0.78846154  0.80288462]
mean accuracy: 0.80462445564
F1-SCORE
[ 0.7020649   0.75274725  0.70769231  0.55232558  0.63661972  0.52727273
  0.60983607  0.60815047  0.58715596  0.63188406]
mean f1: 0.631574904124


ACCURACY
[ 0.83067093  0.86421725  0.8464      0.7632      0.7888      0.7472
  0.8032      0.8032      0.78525641  0.79326923]
mean accuracy: 0.802541381994
F1-SCORE
[ 0.66875     0.75138122  0.69811321  0.54819277  0.62678063  0.50595238
  0.58503401  0.6025641   0.56962025  0.60188088]
mean f1: 0.615826944891


ACCURACY
[ 0.80990415  0.81629393  0.8624      0.7952

# Post processing

In [9]:
from skimage.morphology import *
def postprocessing(Z, w, h, patch_size):
    n_patch = np.int(w/patch_size)
    Z = Z.reshape(-1,n_patch*n_patch)
    
    Z = np.apply_along_axis(post_image, arr=Z, axis=1)
    Z = Z.reshape(-1,1)
    return Z

def post_image(Z):
    binary_closing(Z, out=Z)
    return Z

# Cross validation with post-processing

In [10]:
from sklearn.model_selection import KFold
def cross_validation(X, Y, mlp, cv=10, post=False, verbose=False):
    tot_f1 = 0
    tot_acc = 0
    kf = KFold(n_splits=cv)

    for k, (train, test) in enumerate(kf.split(X)):
        x_train = X[train]
        x_test = X[test]
        y_train = Y[train]
        y_test = Y[test]

        mlp.fit(x_train,y_train)

        z = mlp.predict(x_test)

        if post:
            z = postprocessing(z, width, height, patch_size)

        Zn = np.nonzero(z)[0]
        Yn = np.nonzero(y_test)[0]

        TPR = len(list(set(Yn) & set(Zn))) / float(len(z))
        f1 = f1_score(y_test,z)
        acc = accuracy_score(y_test,z)

        tot_f1 = tot_f1 + f1
        tot_acc = tot_acc + acc
        
        if verbose:
            print('K: ', k)
            print('True positive rate = ' + str(TPR))
            print('F1-score:', f1)
            print('accuracy score:', acc)
            print('\n')

    avg_f1 = tot_f1/cv
    avg_acc = tot_acc/cv
    
    print('Average Accuracy score:', avg_acc)
    print('Average F1-score:', avg_f1)

In [ ]:
for m in models:
    print()
    cross_validation(m[1],m[2],m[0],post=False)
    cross_validation(m[1],m[2],m[0],post=True)


Average Accuracy score: 0.70864
Average F1-score: 0.268947155534
Average Accuracy score: 0.70752
Average F1-score: 0.286507705869

Average Accuracy score: 0.80592
Average F1-score: 0.63569299633
Average Accuracy score: 0.80144
Average F1-score: 0.655640240535

Average Accuracy score: 0.79792
Average F1-score: 0.611084346614
Average Accuracy score: 0.79872
Average F1-score: 0.641035679034

Average Accuracy score: 0.80544
Average F1-score: 0.606063217682
Average Accuracy score: 0.79696
Average F1-score: 0.598515385617


# Cross-validation for n_neighbors and degree

In [ ]:
X0, Y0 = build_model_data(ROOT_DIR, extract_features_cogrey, patch_size=patch_size, n_img=n_img)
degrees = [2, 3, 5, 7]
n_neighbors = [3, 5, 7, 10]
for d in degrees:
    poly = PolynomialFeatures(d)
    X = poly.fit_transform(X0)
    X = normalize(X)
    scaler = StandardScaler()
    scaler.fit(X)  
    X = scaler.transform(X)

    for n in n_neighbors:
        mlp = MLPClassifier()
        
        cr_val = cross_val_score(mlp, X, Y0, cv=10, scoring='f1')
        print('F1-SCORE')
        print(cr_val)
        print('mean f1:', cr_val.mean())
        print('\n')

Original loaded dataset size: 10


# Predictions

In [ ]:
from sklearn.model_selection import train_test_split

def predict(neigh):
    X, Y = build_model_data(ROOT_DIR, extract_features_edge, patch_size=patch_size, n_img=1000)
    poly = PolynomialFeatures(3)
    X = poly.fit_transform(X)
    X = normalize(X)
    scaler = StandardScaler()
    scaler.fit(X)  
    X = scaler.transform(X)
    mlp = MLPClassifier()
    
    cut = int(0.8*(Y.shape[0]))
    X_train = X[0:cut]
    X_test = X[cut:]
    Y_train = Y[0:cut]
    Y_test = Y[cut:]
    mlp.fit(X_train, Y_train)

    z = mlp.predict(X_test)
    z = postprocessing(z, width, height, patch_size)
    
    return z, Y_test

In [ ]:
z, Y_test = predict(mlp4)

In [ ]:
pred_size = int(width/patch_size)*int(height/patch_size)
for ix, i in enumerate(range(0,z.shape[0],pred_size)):
    pred = z[i:i+pred_size]
    im = label_to_img(width, height, patch_size, patch_size, pred)
    plt.imsave('post/images/pred_{}'.format(ix), im, cmap='Greys_r')
    
for ix, i in enumerate(range(0,Y_test.shape[0],pred_size)):
    pred = Y_test[i:i+pred_size]
    im = label_to_img(width, height, patch_size, patch_size, pred)
    pimg = Image.fromarray((im*255.0).astype(np.uint8))
    pimg.save('post/groundtruth/pred_{}.png'.format(ix))

In [ ]:
XX, YY = build_model_data(extract_features_2d, patch_size=patch_size, n_img=n_img)